# Case 4: programming
Use programming tools (python within Jupyter Notebooks) to look at the data.

## About Notebooks

### Inside
A notebook consists of cells. You can write code inside code-cells. That's the cells with an `In [ ]:` to their left.
You run it by pressing `SHIFT + ENTER`.

You can also make text-cells. For formatting, the Markdown language can be used. It is not extremely sofisticated. A header line starts with `#`. A sub-header starts with `##`. Etc.

By clicking outside a cell (or pressing `ESC`) you get out of it. Then you can change the cell type using the appropriate magic key combination:

`m`: make it a text cell

`y`: make it a code cell

### Outside
On top you see a tool-bar. There you can find options for restarting your kernel (basically resetting everything) and running cells (if you dont like pressing `SHIFT + ENTER`), renaming your notebook, downloading it, and many others.

# 0. Gentle introduction to Python
(you can skip this if you like)

## 0.1 The basics

In [ ]:
print("Hello World!")

In [ ]:
# f-strings: put an f in front and then you can print parametres by putting them between curly braces.
name = "Bruce Lee"
print(f"Hello, {name}!")

In [ ]:
# a minimalistic loop
my_number = 0
my_numbers_as_string = ""
for j in range(10):
    my_number = my_number + j
    my_numbers_as_string = my_numbers_as_string + " + " + str(j)
    
my_numbers_as_string = my_numbers_as_string + " = "
print(my_numbers_as_string, my_number)

In [ ]:
# lists are simple containers declared using brackets
my_list = [7, 6, 4]

# dictionaries are nice containers you can make using curly braces
my_dict = {"one": 1, "two": 22, "three": my_list}

print(my_dict)

In [ ]:
# look-up using the key-name
print(my_dict["three"])

## 0.2 Using libraries

In [ ]:
import numpy as np

In [ ]:
x = np.linspace(0.0, 2.0, 20)
print(f"my x-variable has {len(x)} floats inside:\n")
print(x)

In [ ]:
y = np.sin(x)
print("now I also have an y-variable where each item is the sin of the corresponding item in the x:\n")
print(y)

In [ ]:
# importing a plotting library and using a magic function (%) that allows that the figures appear inside the book.
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# very basic plot
ax = plt.plot(x,y)

In [ ]:
# a bit more advanced plot
fig, ax = plt.subplots()
ax.plot(x, y, "o-r")
ax.set_xlim((0.2, 1.2))
ax.set_xlabel("x-value (a.u.)")
ax.set_ylabel("y-value (a.u.)")

In [ ]:
import pandas as pd

In [ ]:
# you can get some information about the stuff you imported by adding a question-mark at the end
pd?

In [ ]:
df = pd.DataFrame({"point": [1,2,3], "height": [17.4, 12.4, 8.1], "model": ["az", "tr", "x12"]})
print(df)

In [ ]:
# jupyter-notebook-trick: just write the variable as the only item on the last line in the cell and it will print.
df

In [ ]:
df.height

In [ ]:
df[["height", "model"]]

# 1. Time to start the show: this is Case 4
What we would like you to do is to load data from two cell-tests and compare them. To do that, you will have to use a library that can read the data-files (cellpy). We recommend that you just run through each cell step-by-step first trying to understand what happens, but without modifying them. Then, to solve the "task" at the bottom, you will have to do some copy-pasting and gentle editing. When you start to feel comfortable with the code snippets and the form of the notebook, feel free to start to modify the content of the cells. For example, you can see what happens if you replace the number `15` in the `.get_cap` cell below with a list of numbers, *e.g.* `[1, 10, 100]`.

**This is what we know about the cells** (from the log)

We have two cells. Both are half-cells with amorphous Si thin-films as active material and Li metal as counter electrode. One has been cycled with using FEC as electrolyte additive. For both cells, the formation was done by cycling at slow rate for 6 cycles.
1. `20150204_TF5_CUA_08_cc`: standard LP30 electrolyte
2. `20150430_TF5_CUA_13_cc`: LP30 + FEC

FEC = Fluoroethylene carbonate

## 1.1 Importing `cellpy` for help in loading the cell data
and other libraries if needed

In [ ]:
# Cellpy will warn you about missing configurations etc. So, lets turn off all warnings this time...
import warnings
warnings.filterwarnings("ignore")
import cellpy
from cellpy import cellreader

# also import the plotting library (in case you skipped the first part (intro to Python))
import matplotlib.pyplot as plt
%matplotlib inline

**Loading the data**

In [ ]:
file_01 = "raw_data/20150204_TF5_CUA_08_cc.h5"
file_02 = "raw_data/20150430_TF5_CUA_13_cc.h5"

In [ ]:
cell_01 = cellreader.CellpyData().load(file_01)
cell_02 = cellreader.CellpyData().load(file_02)

## 1.2 Start looking into what we have
### voltage-capacity cycles

In [ ]:
cycle_15_cell_01 = cell_01.get_cap(15)

In [ ]:
# the object returned by the method .get_cap is a pandas DataFrame. This is a "short-cut" way for plotting it:
cycle_15_cell_01.plot(x="capacity", y="voltage")

### crating dqdv

In [ ]:
# create a custom function for making dqdv
from cellpy.utils import ica
import pandas as pd

def make_dqdv(cell, cycle_number):
    voltage_cycle = cell.get_cap(cycle_number, method="forth-and-forth", categorical_column=True)
    ica_x, ica_y = ica.dqdv_cycle(voltage_cycle)
    frame = pd.DataFrame({"voltage": ica_x, "dqdv": ica_y})
    return frame
    

In [ ]:
dqdv_15_cell_01 = make_dqdv(cell_01, 15)

In [ ]:
dqdv_15_cell_01.plot(x="voltage", y="dqdv")

In [ ]:
# get summary data for cell_01
all_summaries_cell_01 = cell_01.dataset.dfsummary.reset_index()

In [ ]:
# lets have a brief look at it (if you want to know the name of all the columns, you can issue print(all_summaries_cell_01.columns))
all_summaries_cell_01.head()

In [ ]:
charge_cap_01 = all_summaries_cell_01["Charge_Capacity(mAh/g)"]
discharge_cap_01 = all_summaries_cell_01["Discharge_Capacity(mAh/g)"]
cycle_number_01 = all_summaries_cell_01["Cycle_Index"]

In [ ]:
# plotting capacity vs cycle number

In [ ]:
fig, ax = plt.subplots()
ax.plot(cycle_number_01, charge_cap_01, 'o-', label="charge")
ax.plot(cycle_number_01, discharge_cap_01, 's-r', label="discharge")
ax.set_xlabel("cycle number")
ax.set_ylabel("capacity (mAh/g(a.m.))")
ax.legend()

## 1.3 Compare charge capacity *vs* cycle number for both cells

This is your assignement!

1. You will have to get the summary data for the second cell (pick the summary data)
2. Then you will have to select the cycle-index column and the charge capacity column for the summary data for the second cell (select the colums).
3. Then you must plot the charge-capacity vs cycle-index for the first cell and for the second cell (create the figure).


In [ ]:
# pick the summary data
all_summaries_cell_02 = cell_02.dataset.dfsummary.reset_index()

In [ ]:
# select the columns

In [ ]:
# create the figure (including both cells)

# 2. What did we find out?

*1. What is the best cell?*  
*2. For how many cycles did each cell live?*  
*3. Is adding FEC a good thing?*

1:

2:

3: